In [1]:
!pip install keras

You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import pandas as pd
import numpy as np

In [3]:
data=pd.read_csv("equations-all-main.csv")

In [4]:
chars = sorted(list({l for word in data["Math"][:] for l in word}))

In [5]:
chars.append("?")

In [6]:
mapping = dict((c, i) for i, c in enumerate(chars))

In [7]:
# Drop rows with any empty cells
data.dropna(axis=0, how='any', thresh=None, subset=None, inplace=True)

In [8]:
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from numpy import array
from pickle import dump
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import LSTM, Dense, Bidirectional, BatchNormalization, Dropout
from keras import optimizers
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

In [58]:
testing = pd.read_csv("equations1.csv")

In [59]:
test_eqs=testing.math[:]
test_sequences = list()
for eq in test_eqs:
	# integer encode line
	encoded_seq = [mapping[char] for char in eq]
	# store
	test_sequences.append(encoded_seq)

In [60]:
test_padded = pad_sequences(test_sequences, padding='post',value=mapping["?"],maxlen=50)
test_sequences = [to_categorical(x, num_classes=128) for x in test_padded]
X_predict = np.array(test_sequences)

In [12]:
class_names=["Unknown","Simple","Linear Equation for degree one","Fraction"]

In [52]:
data_per_classes_under_sampled["Fraction"].head()

,Math,Class
13445,5\ \frac{1}{4}+3\ \frac{1}{5}+3\ \frac{1}{2=},Fraction
12178,5 \cdot\frac{5}{6}$$ and,Fraction
8895,2^2+\ \frac{1}{6},Fraction
7467,225\ \frac{1}{^251\sqrt{5\ \frac{1}{\sqrt{711}}}},Fraction
7680,23\times\frac{2}{8},Fraction


In [13]:
data_per_classes=dict()
for c in class_names:
   data_per_classes[c]=data[data['Class'] == c]


In [14]:
data_per_classes_under_sampled=dict()
for c in class_names:
   data_per_classes_under_sampled[c]=data_per_classes[c].sample(8000,replace=True)

In [15]:
data_test_under = pd.concat([data_per_classes_under_sampled[c] for c in class_names], axis=0)

print('Random under-sampling:')
print(data_test_under.Class.value_counts())

data_test_under.Class.value_counts().plot(kind='bar', title='Count (target)');

Random under-sampling:
Simple                            8000
Unknown                           8000
Linear Equation for degree one    8000
Fraction                          8000
Name: Class, dtype: int64


In [16]:
def get_X_Y(data):
  equations=data["Math"][:]
  sequences = list()
  for eq in equations:
    # integer encode line
    encoded_seq = [mapping[char] for char in eq]
    # store
    sequences.append(encoded_seq)
  padded = pad_sequences(sequences, padding='post',value=mapping["?"],maxlen=50)
  classes=np.array(data.Class[:])
  label_encoder = LabelEncoder()
  integer_encoded = label_encoder.fit_transform(classes)
  sequences = padded
  total_classes=len(np.unique(data.Class))
  sequences = [to_categorical(x, num_classes=128) for x in sequences]
  X = np.array(sequences)
  y = to_categorical(integer_encoded, num_classes=4)
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.02)
  return  X_train, X_test, y_train, y_test,total_classes


In [17]:
X_train, X_test, y_train, y_test,total_classes=get_X_Y(data_test_under)

In [18]:
# define model
model = Sequential()
model.add(LSTM(250,input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.4))
model.add(Dense(100, activation="relu"))
model.add(Dense(100, activation="relu"))
model.add(Dense(len(class_names), activation='softmax'))
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 250)               379000    
_________________________________________________________________
dropout_1 (Dropout)          (None, 250)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               25100     
_________________________________________________________________
dense_2 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_3 (Dense)              (None, 4)                 404       
Total params: 414,604
Trainable params: 414,604
Non-trainable params: 0
_________________________________________________________________
None


In [19]:
from keras.callbacks import ModelCheckpoint

In [20]:
filepath="weights-improvement-{epoch:02d}-{val_accuracy:.2f}.h5"

In [21]:
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [22]:
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model

In [23]:
history = model.fit(X_train, y_train, validation_split=0.1,epochs=30, callbacks=callbacks_list)

W1108 11:03:17.746377 140501295167296 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.



Train on 28224 samples, validate on 3136 samples
Epoch 1/30
28224/28224 [==============================] - 45s 2ms/step - loss: 0.9063 - accuracy: 0.4863 - val_loss: 0.8464 - val_accuracy: 0.4923

Epoch 00001: val_accuracy improved from -inf to 0.49235, saving model to weights-improvement-01-0.49.h5
Epoch 2/30
28224/28224 [==============================] - 44s 2ms/step - loss: 0.8548 - accuracy: 0.5035 - val_loss: 0.8386 - val_accuracy: 0.5105

Epoch 00002: val_accuracy improved from 0.49235 to 0.51052, saving model to weights-improvement-02-0.51.h5
Epoch 3/30
28224/28224 [==============================] - 44s 2ms/step - loss: 0.8749 - accuracy: 0.5043 - val_loss: 0.8399 - val_accuracy: 0.5022

Epoch 00003: val_accuracy did not improve from 0.51052
Epoch 4/30
28224/28224 [==============================] - 44s 2ms/step - loss: 0.8062 - accuracy: 0.5606 - val_loss: 0.6758 - val_accuracy: 0.7331

Epoch 00004: val_accuracy improved from 0.51052 to 0.73310, saving model to weights-improveme

In [24]:
accr = model.evaluate(X_test,y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

640/640 [==============================] - 1s 866us/step
Test set
  Loss: 0.040
  Accuracy: 0.986


In [61]:
y_pred=model.predict_classes(X_predict)

In [62]:
y_pred

array([2, 0, 1, 3, 2, 0, 3, 2, 3, 2, 3, 2, 1, 3, 2, 2, 2, 1, 2, 2, 3, 1,
       3, 2, 2, 3, 3, 3, 3, 1, 2, 3, 2, 3, 1, 3, 2, 2, 2, 2, 1, 2, 3, 3,
       2, 1, 3, 2, 3, 3, 1, 0, 2, 1, 2, 3, 2, 3, 3, 3, 2, 3, 3, 3, 3, 0,
       2, 3, 2, 3, 2, 3, 3, 2, 3, 2, 3, 2, 2, 3, 2, 1, 2, 2, 3, 2, 2, 3,
       2, 1, 2, 3, 2, 3, 2, 2, 3, 0, 2, 2, 2, 0, 3, 2, 3, 3, 2, 2, 2, 2,
       2, 2, 2, 3, 3, 3, 1, 2, 3, 3, 2, 0, 3, 2, 3, 2, 1, 1, 2, 1, 3, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 3, 2, 2, 2, 1, 3, 2, 3, 2, 3, 2, 1,
       2, 2, 2, 3, 2, 3, 2, 3, 1, 2, 3, 2, 3, 3, 1, 3, 2, 2, 2, 2, 3, 3,
       2, 3, 2, 2, 3, 3, 3, 2, 2, 0, 2, 2, 3, 2, 1, 2, 2, 2, 2, 2, 2, 3,
       2, 2, 1, 1, 2, 2, 1, 3, 2, 3, 2, 2, 2, 3, 2, 2, 2, 2, 3, 3, 3, 2,
       3, 3, 3, 3, 3, 2, 1, 3, 3, 3, 3, 2, 2, 2, 0, 3, 2, 2, 1, 2, 2, 2,
       2, 2, 2, 1, 2, 2, 3, 3, 3, 2, 3, 2, 3, 1, 2, 3, 2, 2, 2, 2, 2, 3,
       3, 3, 2, 2, 3, 2, 1, 2, 2, 2, 2, 3, 3, 1, 0, 3, 3, 2, 3, 1, 3, 3,
       2, 1, 2, 3, 0, 2, 2, 3, 3, 3, 3, 3, 2, 1, 3,

In [63]:
type(np.array(test_eqs))

numpy.ndarray

In [29]:
type(y_pred)

numpy.ndarray

In [32]:
y_pred[:]

array([2, 0, 1, 3, 2, 0, 3, 2, 3, 2, 3, 2, 1, 3, 2, 2, 2, 1, 2, 2, 3, 1,
       3, 2, 2, 3, 3, 3, 3, 1, 2, 3, 2, 3, 1, 3, 2, 2, 2, 2, 1, 2, 3, 3,
       2, 1, 3, 2, 3, 3, 1, 0, 2, 1, 2, 3, 2, 3, 3, 3, 2, 3, 3, 3, 3, 0,
       2, 3, 2, 3, 2, 3, 3, 2, 3, 2, 3, 2, 2, 3, 2, 1, 2, 2, 3, 2, 2, 3,
       2, 1, 2, 3, 2, 3, 2, 2, 3, 0, 2, 2, 2, 0, 3, 2, 3, 3, 2, 2, 2, 2,
       2, 2, 2, 3, 3, 3, 1, 2, 3, 3, 2, 0, 3, 2, 3, 2, 1, 1, 2, 1, 3, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 3, 2, 2, 2, 1, 3, 2, 3, 2, 3, 2, 1,
       2, 2, 2, 3, 2, 3, 2, 3, 1, 2, 3, 2, 3, 3, 1, 3, 2, 2, 2, 2, 3, 3,
       2, 3, 2, 2, 3, 3, 3, 2, 2, 0, 2, 2, 3, 2, 1, 2, 2, 2, 2, 2, 2, 3,
       2, 2, 1, 1, 2, 2, 1, 3, 2, 3, 2, 2, 2, 3, 2, 2, 2, 2, 3, 3, 3, 2,
       3, 3, 3, 3, 3, 2, 1, 3, 3, 3, 3, 2, 2, 2, 0, 3, 2, 2, 1, 2, 2, 2,
       2, 2, 2, 1, 2, 2, 3, 3, 3, 2, 3, 2, 3, 1, 2, 3, 2, 2, 2, 2, 2, 3,
       3, 3, 2, 2, 3, 2, 1, 2, 2, 2, 2, 3, 3, 1, 0, 3, 3, 2, 3, 1, 3, 3,
       2, 1, 2, 3, 0, 2, 2, 3, 3, 3, 3, 3, 2, 1, 3,

In [64]:
list(test_eqs)

['10\\times4\\times8',
 '\\frac{ds}{dt}',
 '5m 78',
 'b3ff',
 '4722\\div22',
 '\\ \\frac{1}{5}(10x-25)',
 '.',
 '22',
 '\\alpha2',
 '67652678+6765',
 'n2+3h2>2nh3',
 '1520x1',
 '6m=12',
 '\\text{despise}',
 '2',
 '(1+3)',
 '2',
 'x=-4',
 '^{ }F+V-E=2',
 '26',
 '346+-777=',
 '7+3\\div(9+x)',
 '.',
 '639=3645.',
 '35\\times2\\div25',
 '10/',
 '(',
 'unsecelentific',
 '60student',
 'x^{2}-11x-12',
 '683',
 '3542\\div63=',
 '29642',
 '. ( 150 - 200 )',
 '3x^2-5x-2=0',
 'm',
 '3',
 '\\ldots',
 '5',
 '5/7 8',
 '3(x+2',
 '137',
 'using',
 '..',
 '9',
 '1/2x+7c=1',
 '.',
 '95012',
 '(7+43) (7',
 '\\text{the}',
 'x^{2}-188+56',
 '\\frac{1}{10}\\div\\frac{-8}{4}',
 '5',
 '3x',
 '78',
 '7sin',
 '2',
 '8/',
 '.',
 '5in25-',
 '8',
 'b) 4',
 '+-10)',
 '\\&',
 '.',
 '\\frac{89728693}{2479}',
 '1',
 '2angles',
 '2',
 '.',
 '8',
 '300+5+',
 '(',
 '2x2',
 '.',
 '444',
 'mukamsinh',
 '63',
 '0',
 '(6x-31x',
 '7',
 '2x+4=6',
 '3/16+13/16',
 '50x5=',
 'karapatang',
 '7+2',
 '8x10',
 'a)',
 '-52',
 'x-7=10'

In [65]:
y_pred_class=[class_names[3-y] for y in y_pred]

In [66]:
y_pred_class

['Simple',
 'Fraction',
 'Linear Equation for degree one',
 'Unknown',
 'Simple',
 'Fraction',
 'Unknown',
 'Simple',
 'Unknown',
 'Simple',
 'Unknown',
 'Simple',
 'Linear Equation for degree one',
 'Unknown',
 'Simple',
 'Simple',
 'Simple',
 'Linear Equation for degree one',
 'Simple',
 'Simple',
 'Unknown',
 'Linear Equation for degree one',
 'Unknown',
 'Simple',
 'Simple',
 'Unknown',
 'Unknown',
 'Unknown',
 'Unknown',
 'Linear Equation for degree one',
 'Simple',
 'Unknown',
 'Simple',
 'Unknown',
 'Linear Equation for degree one',
 'Unknown',
 'Simple',
 'Simple',
 'Simple',
 'Simple',
 'Linear Equation for degree one',
 'Simple',
 'Unknown',
 'Unknown',
 'Simple',
 'Linear Equation for degree one',
 'Unknown',
 'Simple',
 'Unknown',
 'Unknown',
 'Linear Equation for degree one',
 'Fraction',
 'Simple',
 'Linear Equation for degree one',
 'Simple',
 'Unknown',
 'Simple',
 'Unknown',
 'Unknown',
 'Unknown',
 'Simple',
 'Unknown',
 'Unknown',
 'Unknown',
 'Unknown',
 'Fraction',

In [67]:
df = pd.DataFrame(list(zip(list(test_eqs), y_pred_class)), 
               columns =['Math', 'Class']) 

In [68]:
export_csv = df.to_csv (r'prediction_equations1_actual.csv', index = None, header=True)